In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
#imdb dataset
data = tfds.load('imdb_reviews',as_supervised=True)
train_data, test_data = data['train'],data['test']
batch_size = 32
train_data = train_data.shuffle(10000)
train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZVRZJW/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZVRZJW/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZVRZJW/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
example, label = next(iter(train_data))
print('Text:\n', example.numpy()[0])
print('\nLabel: ', label.numpy()[0])

Text:
 b'I admit to being somewhat jaded about the movie genre of a young child softening the heart of his/her reluctant guardian. I\'ve seen enough of them \xc2\x97 Baby Boom, Kolya, About a Boy, Mostly Martha, and to some extent, Whale Rider \xc2\x97 to expect to be bored by the formula. What held my attention in The King of Masks was the grimness of the setting: small-town China in the 1930\'s. Extreme poverty was the norm, and girl children were considered so worthless to poor parents that they killed them at birth or gave them to whomever would take them on the black market. When Wang discovers his purchased grandson, whom he\'s nicknamed "Doggie," is a granddaughter, he initially casts her out, even though she\'s showed great promise as street-performing heir. Even after he reluctantly takes her back, he\'s not too upset when she\'s kidnapped. The film is gritty then, showing the lengths to which a young, street-smart girl had to go to survive in that society.<br /><br />The two 

In [ ]:
# Using the TextVectorization layer to normalize, split, and map strings
# to integers.
encoder = tf.keras.layers.TextVectorization(max_tokens=10000)
encoder.adapt(train_data.map(lambda text, _: text))

# Extracting the vocabulary from the TextVectorization layer.
import numpy as np
vocabulary = np.array(encoder.get_vocabulary())

# Encoding a test example and decoding it back.
original_text = example.numpy()[0]
encoded_text = encoder(original_text).numpy()
decoded_text = ' '.join(vocabulary[encoded_text])

print('original: ', original_text)
print('encoded: ', encoded_text)
print('decoded: ', decoded_text)

original:  b'I admit to being somewhat jaded about the movie genre of a young child softening the heart of his/her reluctant guardian. I\'ve seen enough of them \xc2\x97 Baby Boom, Kolya, About a Boy, Mostly Martha, and to some extent, Whale Rider \xc2\x97 to expect to be bored by the formula. What held my attention in The King of Masks was the grimness of the setting: small-town China in the 1930\'s. Extreme poverty was the norm, and girl children were considered so worthless to poor parents that they killed them at birth or gave them to whomever would take them on the black market. When Wang discovers his purchased grandson, whom he\'s nicknamed "Doggie," is a granddaughter, he initially casts her out, even though she\'s showed great promise as street-performing heir. Even after he reluctantly takes her back, he\'s not too upset when she\'s kidnapped. The film is gritty then, showing the lengths to which a young, street-smart girl had to go to survive in that society.<br /><br />The 

In [ ]:
# building  the model
model = tf.keras.Sequential([
	encoder,
	tf.keras.layers.Embedding(
		len(encoder.get_vocabulary()), 64, mask_zero=True),
	tf.keras.layers.Bidirectional(
		tf.keras.layers.LSTM(64, return_sequences=True)),
	tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
	tf.keras.layers.Dense(64, activation='relu'),
	tf.keras.layers.Dense(1)
])
model.summary()
model.compile(
	loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
	optimizer=tf.keras.optimizers.Adam(),
	metrics=['accuracy']
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, None)              0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, None, 64)          640000    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [ ]:
# Training the model
history = model.fit(
	train_data,
	epochs=10,
	validation_data=test_data,
)

Epoch 1/10
782/782 [==============================] - 214s 249ms/step - loss: 0.4135 - accuracy: 0.7945 - val_loss: 0.3685 - val_accuracy: 0.8460
Epoch 2/10
782/782 [==============================] - 142s 182ms/step - loss: 0.2753 - accuracy: 0.8809 - val_loss: 0.3739 - val_accuracy: 0.8476
Epoch 3/10
782/782 [==============================] - 128s 163ms/step - loss: 0.2361 - accuracy: 0.9012 - val_loss: 0.3499 - val_accuracy: 0.8473
Epoch 4/10
782/782 [==============================] - 121s 155ms/step - loss: 0.1532 - accuracy: 0.9420 - val_loss: 0.3435 - val_accuracy: 0.8610
Epoch 5/10
782/782 [==============================] - 119s 152ms/step - loss: 0.1007 - accuracy: 0.9639 - val_loss: 0.4445 - val_accuracy: 0.8606
Epoch 6/10
782/782 [==============================] - 114s 146ms/step - loss: 0.0713 - accuracy: 0.9762 - val_loss: 0.4595 - val_accuracy: 0.8494
Epoch 7/10
782/782 [==============================] - 115s 147ms/step - loss: 0.0540 - accuracy: 0.9817 - val_loss: 0.5679 -

In [ ]:
# Making predictions
sample_text = (
	'''I didnt like the biriyani that i had from abc restaurant.''')
predictions = model.predict(np.array([sample_text]))
print(*predictions[0])
if predictions[0] > 0:
	print('positive review')
else:
	print('negative review')

1/1 [==============================] - 0s 21ms/step
-4.389141
The review is negative
